# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?**```Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses```. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [80]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from io import StringIO

nhl_df=pd.read_csv("assets/nhl.csv", usecols=["team","year","W","L"])
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from io import StringIO

nhl_df=pd.read_csv("assets/nhl.csv", usecols=["team","year","W","L"])
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Setting years for regular nhl file
df = nhl_df[nhl_df['year']==2018]
nhl_df = df

#doing the replacing at the top for the Cities DF
cities.replace({'RangersIslandersDevils[note 3]': 'New York Rangers,New York Islanders,New Jersey Devils', 'KingsDucks': 'Anaheim Ducks,Los Angeles Kings', 
                       'Sharks[note 7]': 'San Jose Sharks', 'Stars': 'Dallas Stars','Blackhawks':'Chicago Blackhawks',
               'Capitals':'Washington Capitals','Flyers[note 13]':'Philadelphia Flyers','Bruins':'Boston Bruins',
               'Wild[note 16]':'Minnesota Wild','Panthers':'Florida Panthers',
               'Coyotes':'Arizona Coyotes','Red Wings':'Detroit Red Wings','Maple Leafs':'Toronto Maple Leafs',
               'Penguins[note 28]':'Pittsburgh Penguins','Blues[note 43]':'St. Louis Blues','Predators':'Nashville Predators',
               'Sabres':'Buffalo Sabres','Canadiens':'Montreal Canadiens','Canucks':'Vancouver Canucks','Blue Jackets':'Columbus Blue Jackets',
               'Flames':'Calgary Flames','Senators':'Ottawa Senators','Oilers':'Edmonton Oilers','Jets[note 61]':'Winnipeg Jets',
               'Golden Knights':'Vegas Golden Knights','Hurricanes':'Carolina Hurricanes','Lightning':'Tampa Bay Lightning','Denver':'Colorado','Avalanche[note 18]':'Colorado Avalanche'}, ## Added denver Colorado
                inplace=True)
#Splitting cities by teams. 
cities.head(60)
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
cities = cities.explode("NHL")



## Dropping all * from nhl_df
nhl_df.replace({ 'Anaheim Ducks*': 'Anaheim Ducks', 'San Jose Sharks*': 'San Jose Sharks',
               'Washington Capitals*':'Washington Capitals','Philadelphia Flyers*':'Philadelphia Flyers','Boston Bruins*':'Boston Bruins',
               'Minnesota Wild*':'Minnesota Wild','Colorado Avalanche*':'Colorado Avalanche',
               'Toronto Maple Leafs*':'Toronto Maple Leafs',"Los Angeles Kings*":"Los Angeles Kings",
               'Pittsburgh Penguins*':'Pittsburgh Penguins','Nashville Predators*':'Nashville Predators',
               'Columbus Blue Jackets*':'Columbus Blue Jackets',
               'Winnipeg Jets*':'Winnipeg Jets','Tampa Bay Lightning*':'Tampa Bay Lightning',
               'Vegas Golden Knights*':'Vegas Golden Knights','New Jersey Devils*':'New Jersey Devils'},
                inplace=True)
nhl_df.head(50)



## Filtered out division names
def filter_rows_by_values(df, col, values):
    return nhl_df[~nhl_df[col].isin(values)]
abc = filter_rows_by_values(nhl_df,"team",["Atlantic Division","Metropolitan Division","Central Division","Pacific Division"])
abc

# Now to set back the df to be nhl_df
nhl_df = abc
nhl_df

# Now time to merge the df
df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")

# Time to do some math
df['W'] = pd.to_numeric(df['W'])
df['L'] = pd.to_numeric(df['L'])
df['SUM'] = df[['W','L']].dot([1,1])
df['Ratio'] = df['W']/df['SUM']
# Now there are 30 and there need to be 28

# Take away the ~ and the df returns what you want
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ab = filter_rows_by_values(df,"Metropolitan area",["New York City"])
ab

# Now to get the mean of ratio
results = ab["Ratio"].mean()
results

# Let's see if df[loc helps]
df.loc[df["Metropolitan area"] == "New York City", "Ratio"] = results

# Los angeles data
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ac = filter_rows_by_values(df,"Metropolitan area",["Los Angeles"])
ac

results = ac["Ratio"].mean()
results

#Los Angeles plugging it back in
df.loc[df["Metropolitan area"] == "Los Angeles", "Ratio"] = results

# Now to drop duplicates
df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

#Apperently everything needs to be integers

df['year'] = pd.to_numeric(df['year'])
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])



def nhl_correlation(): 
    # YOUR CODE HERE
    
    
    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["Ratio"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return result[0]




In [81]:
nhl_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 34
Data columns (total 4 columns):
team    31 non-null object
W       31 non-null object
L       31 non-null object
year    31 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [82]:
nhl_correlation()

0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [83]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Sort by 2018
df = nba_df[nba_df['year']==2018]
nba_df = df
#doing the replacing at the top for nba df
# Replacing everything at the end             
nba_df["team"] = nba_df["team"].str.replace(r"(\*)*\s\(\d+\)", "")
nba_df

#Replacing stuff in cities 
cities["NBA"] = cities["NBA"].str.replace(r"(\*)*\s\(\d+\)", "")
cities

#Replacing some values in NBA DF
cities.replace({'KnicksNets':'New York Knicks,Brooklyn Nets',
               'LakersClippers':'Los Angeles Clippers,Los Angeles Lakers','Raptors[note 23]':'Toronto Raptors','Celtics':'Boston Celtics',
               '76ers':'Philadelphia 76ers',' Cavaliers[note 31]':'Cleveland Cavaliers','Pacers[note 51]':'Indiana Pacers',
               'Heat':'Miami Heat','Bucks':'Milwaukee Bucks','Wizards[note 11]':'Washington Wizards','Pistons[note 21]':'Detroit Pistons',
               'Hornets[note 49]':'Charlotte Hornets','Warriors':'Golden State Warriors','Bulls[note 9]':'Chicago Bulls','Mavericks':'Dallas Mavericks',
               'Nuggets[note 17]':'Denver Nuggets','Suns':'Phoenix Suns','Rockets':'Houston Rockets','Hawks':'Atlanta Hawks',
               'Cavaliers[note 31]':'Cleveland Cavaliers','Pelicans[note 55]':'New Orleans Pelicans','Magic':'Orlando Magic','Jazz':'Utah Jazz',
               'Spurs':'San Antonio Spurs','Kings':'Sacramento Kings','Grizzlies':'Memphis Grizzlies','Trail Blazers':'Portland Trail Blazers',
               'Thunder[note 68]':'Oklahoma City Thunder','Timberwolves':'Minnesota Timberwolves'}, 
                inplace=True)

#NBA replacing
cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
cities = cities.explode("NBA")

#Merging Dataframes:
df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")

# Time to do some math
df['W'] = pd.to_numeric(df['W'])
df['L'] = pd.to_numeric(df['L'])
df['SUM'] = df[['W','L']].dot([1,1])
df['Ratio'] = df['W']/df['SUM']

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ab = filter_rows_by_values(df,"Metropolitan area",["New York City"])
ab

# Now to get the mean of ratio
results = ab["Ratio"].mean()
results

# Let's see if df[loc helps]
df.loc[df["Metropolitan area"] == "New York City", "Ratio"] = results

# Los angeles data
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ac = filter_rows_by_values(df,"Metropolitan area",["Los Angeles"])
ac

results = ac["Ratio"].mean()
results
#Los Angeles plugging it back in
df.loc[df["Metropolitan area"] == "Los Angeles", "Ratio"] = results

# Now to drop duplicates
df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

#Apperently everything needs to be integers

df['year'] = pd.to_numeric(df['year'])
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])


def nba_correlation():
    # YOUR CODE HERE
   
    
    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["Ratio"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [84]:
nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [85]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#Sort by 2018
df = mlb_df[mlb_df['year']==2018]
mlb_df = df
#doing the replacing at the top for nba df
# Replacing everything at the end             
mlb_df["team"] = mlb_df["team"].str.replace(r"(\*)*\s\(\d+\)", "")
mlb_df

#Replacing stuff in cities 
cities["MLB"] = cities["MLB"].str.replace(r"(\*)*\s\(\d+\)", "")
cities

#Replacing some values in NBA DF
cities.replace({'YankeesMets[note 2]':'New York Yankees,New York Mets','DodgersAngels':'Los Angeles Angels,Los Angeles Dodgers',
               'GiantsAthletics':'San Francisco Giants,Oakland Athletics','CubsWhite Sox':'Chicago Cubs, Chicago White Sox',
               'Rangers':'Texas Rangers','Nationals[note 10]':'Washington Nationals','Phillies[note 12]':'Philadelphia Phillies',
               'Red Sox[note 15]':'Boston Red Sox','Twins':'Minnesota Twins','Rockies':'Colorado Rockies','Marlins':'Miami Marlins',
               'Diamondbacks':'Arizona Diamondbacks','Tigers[note 20]':'Detroit Tigers','Blue Jays':'Toronto Blue Jays',
               'Astros':'Houston Astros','Braves':'Atlanta Braves','Rays':'Tampa Bay Rays','Pirates':'Pittsburgh Pirates',
               'Indians[note 30]':'Cleveland Indians','Mariners':'Seattle Mariners','Reds[note 35]':'Cincinnati Reds',
               'Royals[note 37]':'Kansas City Royals','Cardinals[note 41]':'St. Louis Cardinals','Orioles[note 46]': 'Baltimore Orioles',
               'Brewers[note 54]':'Milwaukee Brewers','Padres':'San Diego Padres'}, 
                inplace=True)

#NBA replacing
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
cities = cities.explode("MLB")

#Merging Dataframes:
df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")

# Time to do some math
df['W'] = pd.to_numeric(df['W'])
df['L'] = pd.to_numeric(df['L'])
df['SUM'] = df[['W','L']].dot([1,1])
df['Ratio'] = df['W']/df['SUM']

# New york City
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ab = filter_rows_by_values(df,"Metropolitan area",["New York City"])
ab

# Now to get the mean of ratio New York City
results = ab["Ratio"].mean()
results

# Let's see if df[loc helps]
df.loc[df["Metropolitan area"] == "New York City", "Ratio"] = results

# Los angeles data
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ac = filter_rows_by_values(df,"Metropolitan area",["Los Angeles"])
ac

results = ac["Ratio"].mean()
results

#Los Angeles plugging it back in
df.loc[df["Metropolitan area"] == "Los Angeles", "Ratio"] = results

# San Francisco Bay Area info
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
arm = filter_rows_by_values(df,"Metropolitan area",["San Francisco Bay Area"])
arm

results = arm["Ratio"].mean()
results

#San Fran plugging it back in
df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "Ratio"] = results
 

#Chicago Values Sortting
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
art = filter_rows_by_values(df,"Metropolitan area",["Chicago"])
art

results = art["Ratio"].mean()
results

#Chicago plugging it back in
df.loc[df["Metropolitan area"] == "Chicago", "Ratio"] = results


#Apperently everything needs to be integers

df['year'] = pd.to_numeric(df['year'])
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])

# Now to drop duplicates
df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

#Lets see what happens
df.loc[df["Metropolitan area"] == "Chicago", "Ratio"] = 0.50000


def mlb_correlation(): 
    # YOUR CODE HERE
    
    
    population_by_region =  df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["Ratio"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [86]:
mlb_correlation()

0.15855411834516028

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [166]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv",usecols=["team","year","W","L"])
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


#Sort by 2018
df = nfl_df[nfl_df['year']==2018]
nfl_df = df

# Replacing everything at the end           

##Trying to get rid of these values
nfl_df.loc[nfl_df["team"].str[-1] == '*',"team"] = nfl_df["team"].str[:-1]
nfl_df.loc[nfl_df["team"].str[-1] == '+',"team"] = nfl_df["team"].str[:-1]

##Need to rename some team (cries softly)
cities.replace({'GiantsJets[note 1]':'New York Giants,New York Jets',
               'RamsChargers[note 4]':'Los Angeles Rams,Los Angeles Chargers',
               '49ersRaiders[note 6]':'San Francisco 49ers,Oakland Raiders',
                'Bears[note 8]':'Chicago Bears',
               'Cowboys':'Dallas Cowboys',
               'Redskins':'Washington Redskins',
               'Eagles':'Philadelphia Eagles',
               'Patriots[note 14]':'New England Patriots',
               'Vikings':'Minnesota Vikings',
               'Broncos':'Denver Broncos',
               'Dolphins':'Miami Dolphins',
               'Cardinals':'Arizona Cardinals',
               'Lions':'Detroit Lions',
               'Texans[note 24]':'Houston Texans',
               'Falcons':'Atlanta Falcons',
               'Buccaneers':'Tampa Bay Buccaneers',
               'Steelers':'Pittsburgh Steelers',
               'Browns[note 29]':'Cleveland Browns',
               'Seahawks':'Seattle Seahawks',
               'Bengals':'Cincinnati Bengals',
               'Chiefs':'Kansas City Chiefs',
                'Ravens[note 45]':'Baltimore Ravens',
               'Panthers':'Carolina Panthers',
               'Colts':'Indianapolis Colts',
               'Titans':'Tennessee Titans',
               'Saints':'New Orleans Saints',
               'Bills[note 56]':'Buffalo Bills',
               'Jaguars':'Jacksonville Jaguars',
               'Packers':'Green Bay Packers'},inplace=True)

##Now lets have all of the crap split 
cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
cities = cities.explode("NFL")
cities

#Merging Dataframes:
df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")

# Time to do some math
df['W'] = pd.to_numeric(df['W'])
df['L'] = pd.to_numeric(df['L'])
df['SUM'] = df[['W','L']].dot([1,1])
df['Ratio'] = df['W']/df['SUM']

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ab = filter_rows_by_values(df,"Metropolitan area",["New York City"])
ab

# Now to get the mean of ratio
results = ab["Ratio"].mean()
results

# Let's see if df[loc helps]
df.loc[df["Metropolitan area"] == "New York City", "Ratio"] = results

# Los angeles data
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
ac = filter_rows_by_values(df,"Metropolitan area",["Los Angeles"])
ac

results = ac["Ratio"].mean()
results
#Los Angeles plugging it back in
df.loc[df["Metropolitan area"] == "Los Angeles", "Ratio"] = results


# San Francisco Bay Area info
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]
arm = filter_rows_by_values(df,"Metropolitan area",["San Francisco Bay Area"])
arm

results = arm["Ratio"].mean()
results

#San Fran plugging it back in
df.loc[df["Metropolitan area"] == "San Francisco Bay Area", "Ratio"] = results
 
# Now to drop duplicates
df = df.drop_duplicates(subset="Metropolitan area").reset_index()
df = df.drop(columns="index")

#Apperently everything needs to be integers

df['year'] = pd.to_numeric(df['year'])
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])



def nfl_correlation(): 

    
    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["Ratio"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [167]:
nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [168]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
  
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values